In [62]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics
from calitp_data_analysis import calitp_color_palette

# from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [3]:
## parameters cell
itp_id = 300

In [4]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt

In [5]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

In [6]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [7]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [8]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [9]:
#  TODO to utils
def time_period_and_arrowize(gdf, time_period):
    
    gdf = gdf >> filter(_.time_of_day == time_period)
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [10]:
peak_test = time_period_and_arrowize(speedmap_segs, 'AM Peak')

In [11]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [12]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
10501,0666caf3ec1ecc96b74f4477ee4bc939,fedc605c108af558e4e545806741ea0f,510358_JUNE24,51-13183,0.0,283__8824,Avalon / Imperial__Avalon / 111th,283-8824-1,AM Peak,12.34,...,12.34,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((160449.331 -452365.708, 160434.047 -...",1,0.333333,51
10502,0666caf3ec1ecc96b74f4477ee4bc939,fedc605c108af558e4e545806741ea0f,510358_JUNE24,51-13183,0.0,8824__8823,Avalon / 111th__Avalon / 110th,8824-8823-1,AM Peak,14.66,...,14.66,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"POLYGON ((160443.819 -452040.084, 160428.472 -...",1,0.333333,51


## export map

In [13]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

NameError: name 'time_of_day' is not defined

In [ ]:
#  TODO clone render method from RtFilterMapper, to utils
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

In [ ]:
speedmap_state

In [17]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'direction_id', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry', 'n_trips_sch',
       'trips_hr_sch', 'route_short_name'],
      dtype='object')

In [ ]:
speedmap_segs

In [15]:
import altair as alt
from vega_datasets import data

source = data.barley()

error_bars = alt.Chart(source).mark_errorbar(extent="stdev").encode(
    alt.Y("yield:Q").scale(zero=False),
    alt.X("variety:N"),
)

points = alt.Chart(source).mark_point(
    filled=True,
    color="black",
).encode(
    alt.Y("mean(yield)"),
    alt.X("variety:N"),
)

error_bars + points

ModuleNotFoundError: No module named 'vega_datasets'

In [34]:
df >> group_by

,time_of_day,p50_mph,p20_mph,p80_mph,y_err,y_err2
5265,AM Peak,13.00,13.00,13.00,13.00,13.00
5266,AM Peak,22.58,22.58,22.58,22.58,22.58
5267,AM Peak,7.31,7.31,7.31,7.31,7.31
5268,AM Peak,15.72,15.72,15.72,15.72,15.72
5269,AM Peak,18.62,18.62,18.62,18.62,18.62
...,...,...,...,...,...,...
126376,Early AM,55.45,55.45,55.45,55.45,55.45
126377,Early AM,54.62,54.62,54.62,54.62,54.62
126378,Early AM,8.22,8.22,8.22,8.22,8.22
126379,Early AM,17.60,17.60,17.60,17.60,17.60


In [48]:
alt.Chart(df).mark_errorbar().encode(
    # y = alt.Y("p50_mph:Q").scale(zero=False),
    y = alt.Y("p50_mph:Q"),
    yError=("y_err:Q"),
    yError2=("y_err2:Q"),
    x = alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl']),
)

alt.Chart(...)

In [51]:
df

,time_of_day,p50_mph,p20_mph,p80_mph,y_err,y_err2
0,AM Peak,13.35,10.74,16.61,2.61,3.26
1,Early AM,15.37,12.85,18.33,2.52,2.96
2,Evening,17.17,15.01,19.62,2.16,2.45
3,Midday,12.78,9.96,16.41,2.82,3.63
4,Owl,19.25,17.13,21.24,2.12,1.99
5,PM Peak,12.78,10.15,16.37,2.63,3.59


In [52]:
-(df['p50_mph'] - df['p20_mph'])

0   -2.61
1   -2.52
2   -2.16
3   -2.82
4   -2.12
5   -2.63
dtype: float64

In [68]:
type(cmap)

branca.colormap.StepColormap

In [70]:
dir(cmap)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_children',
 '_env',
 '_get_self_bounds',
 '_id',
 '_name',
 '_parent',
 '_repr_html_',
 '_template',
 'add_child',
 'add_children',
 'add_to',
 'caption',
 'colors',
 'get_bounds',
 'get_name',
 'get_root',
 'index',
 'render',
 'rgb_bytes_tuple',
 'rgb_hex_str',
 'rgba_bytes_tuple',
 'rgba_floats_tuple',
 'rgba_hex_str',
 'save',
 'scale',
 'to_dict',
 'to_json',
 'to_linear',
 'vmax',
 'vmin']

In [71]:
cmap.get_bounds()

[[None, None], [None, None]]

In [75]:
dir(cmap.scale())

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_children',
 '_env',
 '_get_self_bounds',
 '_id',
 '_name',
 '_parent',
 '_repr_html_',
 '_template',
 'add_child',
 'add_children',
 'add_to',
 'caption',
 'colors',
 'get_bounds',
 'get_name',
 'get_root',
 'index',
 'render',
 'rgb_bytes_tuple',
 'rgb_hex_str',
 'rgba_bytes_tuple',
 'rgba_floats_tuple',
 'rgba_hex_str',
 'save',
 'scale',
 'to_dict',
 'to_json',
 'to_linear',
 'vmax',
 'vmin']

In [69]:
df = speedmap_segs[['time_of_day', 'p50_mph', 'p20_mph', 'p80_mph']]
df = df >> group_by(_.time_of_day) >> summarize(p50_mph = _.p50_mph.quantile(.5),
                                               p20_mph = _.p20_mph.quantile(.5),
                                               p80_mph = _.p80_mph.quantile(.5),)
df['y_err'] = -(df['p50_mph'] - df['p20_mph'])
df['y_err2'] = df['p80_mph'] - df['p50_mph']

error_bars = alt.Chart(df).mark_errorbar(thickness=5,
                                         color='gray',
                                        opacity=.8).encode(
    # y = alt.Y("p50_mph:Q").scale(zero=False),
    y = alt.Y("p50_mph:Q").scale(zero=False),
    yError=("y_err:Q"),
    yError2=("y_err2:Q"),
    x = alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl']),
)

points = alt.Chart(df).mark_point(
    filled=True,
    # color="black",
).encode(
    alt.Y("p50_mph:Q"),
    alt.X("time_of_day:N", sort=['Early AM', 'AM Peak', 'Midday', 'PM Peak', 'Evening', 'Owl']),
    color=alt.Color('p50_mph', scale=cmap)
)

error_bars + points
# error_bars

SchemaValidationError: `Scale` has no parameter named 'children'

Existing parameter names are:
align      domain      exponent       paddingOuter   round    
base       domainMax   interpolate    range          scheme   
bins       domainMid   nice           rangeMax       type     
clamp      domainMin   padding        rangeMin       zero     
constant   domainRaw   paddingInner   reverse                 

See the help for `Scale` to read the full description of these parameters

alt.LayerChart(...)

In [22]:
source = pd.DataFrame({
    "yield_error": [7.5522, 6.9775, 3.9167, 11.9732],
    "yield_center": [32.4, 30.96667, 33.966665, 30.45],
    "variety": ["Glabron", "Manchuria", "No. 457", "No. 462"],
})

bar = alt.Chart(source).mark_errorbar().encode(
    x=alt.X("yield_center:Q").scale(zero=False).title("yield"),
    xError=("yield_error:Q"),
    y=alt.Y("variety:N"),
)

In [23]:
bar

alt.Chart(...)

In [ ]:
## TODO get a better chart -- p20 - 50 - 80 seg speed?
# try:
#     rt_day.set_filter(start_time='06:00', end_time='22:00')
#     display(Markdown(f'Median Trip Speed by Hour of Day{rt_day.filter_formatted}'))

#     rt_day.chart_speeds(no_title=True);
# except:
#     display(Markdown(f'Chart not available for this agency'))
#     pass